# Intermittent Math Modelling

This notebook focuses on mathematical modeling of intermittent treatment data for cancer cell growth dynamics.
Data is extracted from the organized processed datasets with exact seeding densities (20k and 30k).

In [ ]:
# Essential packages for intermittent treatment modeling should only run once per project
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("DifferentialEquations")
Pkg.add("Optimization")
Pkg.add("OptimizationOptimJL")
Pkg.add("OptimizationBBO")
Pkg.add("DiffEqParamEstim")
Pkg.add("StatsBase")
Pkg.add("SciMLSensitivity")
Pkg.add("BlackBoxOptim")
Pkg.add("StatsPlots")
Pkg.add("DataFramesMeta")
Pkg.add("Interact")
Pkg.add("Random")
Pkg.add("Distributions")

    Updating registry at `C:\Users\MainFrameTower\.julia\registries\General.toml`
   Resolving package versions...
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed DataAPI ───────────────────── v1.16.0
   Installed SentinelArrays ────────────── v1.4.8
   Installed PooledArrays ──────────────── v1.4.3
   Installed InlineStrings ─────────────── v1.4.5
   Installed TableTraits ───────────────── v1.0.1
   Installed Tables ────────────────────── v1.12.1
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed DataAPI ───────────────────── v1.16.0
   Installed SentinelArrays ────────────── v1.4.8
   Installed PooledArrays ──────────────── v1.4.3
   Installed InlineStrings ─────────────── v1.4.5
   Installed TableTraits ───────────────── v1.0.1
   Installed Tables ────────────────────── v1.12.1
   Installed DataValueInterfaces ───────── v1.0.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed Compat ────────────────────── v4.18.0
   Installed Compat ──────────

In [ ]:
using CSV
using DataFrames
using Plots
using DifferentialEquations
using Optimization, OptimizationOptimJL, OptimizationBBO
using BlackBoxOptim
using StatsBase
using Random
using Distributions
using Printf
using Statistics

In [ ]:
# Core modeling functions
function setUpProblem(modelTypeSet, xdataSet, ydataSet, solverSet, u0Set, pSet, tspanSet, boundsSet)
    best_params, best_sol, best_prob, best_loss = nothing, nothing, nothing, Inf

    for _ in 1:5
        p_init = [lo[1] == lo[2] ? lo[1] : rand(Uniform(lo[1], lo[2])) for lo in boundsSet]

        try
            prob = ODEProblem(modelTypeSet, u0Set, tspanSet, p_init)
            loss_func = build_loss_objective(prob, solverSet, L2Loss(xdataSet, ydataSet), Optimization.AutoForwardDiff())
            result = bboptimize(loss_func; SearchRange=boundsSet, MaxTime=30.0, TraceMode=:silent)
            p_opt = best_candidate(result)
            sol = solve(remake(prob, p=p_opt), solverSet, saveat=xdataSet)
            loss = sum(abs2.(ydataSet .- [u[1] for u in sol.u]))

            if loss < best_loss
                best_params, best_sol, best_prob, best_loss = p_opt, sol, prob, loss
            end
        catch; end
    end

    return best_params, best_sol, best_prob
end

function calculate_bic(probbic, xdatabic, ydatabic, solverbic, optparbic)
    solbic = solve(probbic, solverbic, reltol=1e-15, abstol=1e-15, saveat=xdatabic)
    residualsbic = [ydatabic[i] - solbic(xdatabic[i])[1] for i in 1:length(xdatabic)]
    ssrbic = sum(residualsbic .^ 2)
    kbic = length(optparbic)
    nbic = length(xdatabic)
    bic = nbic * log(ssrbic / nbic) + kbic * log(nbic)
    return bic, ssrbic
end

function plot_model_fit(x, y, optimized_params, optimized_sol, bic, ssr, title_str)
    println("\nOptimized Parameters: ", round.(optimized_params, digits=4))
    println("Sum of Squared Residuals (SSR): ", round(ssr, digits=6))
    println("Bayesian Information Criterion (BIC): ", round(bic, digits=2))
    
    p = scatter(x, y, label="Data", legend=:bottomright, title=title_str, xlabel="Day", ylabel="Cell Count")
    plot!(optimized_sol.t, [u[1] for u in optimized_sol.u], label="Model", lw=2)
    
    display(p)
    return p
end

In [ ]:
# Load intermittent data from processed datasets
function load_intermittent_data(seeding_density::String)
    base_path = "Processed_Datasets/Intermittent Data/$(seeding_density)_seeding_density/Averages/"
    
    # Define file mappings
    files = [
        "A2780T_dayaverages.csv",     # Naive treated
        "A2780UT_dayaverages.csv",    # Naive untreated  
        "A2780cisT_dayaverages.csv",  # Resistant treated
        "A2780cisUT_dayaverages.csv"  # Resistant untreated
    ]
    
    datasets = Dict{String, DataFrame}()
    
    for file in files
        try
            df = CSV.read(base_path * file, DataFrame)
            datasets[file] = df
            println("✅ Loaded: ", file)
        catch e
            println("❌ Failed to load: $file")
            println("   ↳ Error: ", e)
        end
    end
    
    return datasets
end

# Extract x,y data from day averages CSV
function extract_xy_from_dayaverages(df::DataFrame; scaling_factor=157.7)
    # Assuming the CSV has columns like 'Day' and 'Average'
    x = Float64.(df[:, 1])  # First column (days)
    y = Float64.(df[:, 2]) ./ scaling_factor  # Second column (cell counts), scaled
    return x, y
end

In [ ]:
# Mathematical models for intermittent treatment

# 1. Standard logistic growth
function logistic_growth!(du, u, p, t)
    r, K = p
    du[1] = r * u[1] * (1 - u[1] / K)
end

# 2. Logistic with exponential drug decay
function logistic_exp_decay!(du, u, p, t)
    r, K, drug_effect, decay_rate = p
    N = u[1]
    # Drug effect decreases exponentially over time
    current_drug_effect = drug_effect * exp(-t / decay_rate)
    effective_growth_rate = r * (1 - current_drug_effect)
    du[1] = effective_growth_rate * N * (1 - N / K)
end

# 3. Intermittent treatment model (step function)
function intermittent_treatment!(du, u, p, t)
    r, K, drug_effect, treatment_period, off_period = p
    N = u[1]
    
    # Calculate if we're in treatment or off period
    cycle_time = treatment_period + off_period
    time_in_cycle = mod(t, cycle_time)
    
    # Apply drug effect only during treatment periods
    current_drug_effect = time_in_cycle <= treatment_period ? drug_effect : 0.0
    effective_growth_rate = r * (1 - current_drug_effect)
    
    du[1] = effective_growth_rate * N * (1 - N / K)
end

# 4. Hill function drug response
function logistic_hill_drug!(du, u, p, t)
    r, K, max_effect, hill_coeff, half_time = p
    N = u[1]
    
    # Hill function for drug response over time
    drug_response = max_effect * (t^hill_coeff) / (t^hill_coeff + half_time^hill_coeff)
    effective_growth_rate = r * (1 - drug_response)
    
    du[1] = effective_growth_rate * N * (1 - N / K)
end

In [ ]:
# Load and analyze 20k seeding density data
println("Loading 20k seeding density data...")
datasets_20k = load_intermittent_data("20k")

# Initialize storage for results
results_20k = Dict{String, Any}()
xy_data_20k = Dict{String, Tuple{Vector{Float64}, Vector{Float64}}}()

# Process each dataset
for (filename, df) in datasets_20k
    println("\n📊 Processing: $filename")
    
    # Extract x,y data
    x, y = extract_xy_from_dayaverages(df)
    xy_data_20k[filename] = (x, y)
    
    # Set up common parameters
    solver = Rodas5()
    tspan = (x[1], x[end])
    u0 = [y[1]]
    
    # Determine if treated or untreated
    is_treated = occursin("T_", filename) && !occursin("UT_", filename)
    
    if is_treated
        println("   → Fitting treatment models")
        
        # Try different treatment models
        models_to_try = [
            (logistic_exp_decay!, [(0.0, 2.0), (100.0, 50000.0), (0.0, 0.9), (0.5, 10.0)], "Exponential Decay"),
            (intermittent_treatment!, [(0.0, 2.0), (100.0, 50000.0), (0.0, 0.9), (1.0, 7.0), (1.0, 7.0)], "Intermittent"),
            (logistic_hill_drug!, [(0.0, 2.0), (100.0, 50000.0), (0.0, 0.9), (1.0, 5.0), (1.0, 10.0)], "Hill Response")
        ]
        
        file_results = []
        
        for (model, bounds, name) in models_to_try
            p_init = [mean(b) for b in bounds]
            
            try
                opt_params, opt_sol, opt_prob = setUpProblem(model, x, y, solver, u0, p_init, tspan, bounds)
                
                if opt_params !== nothing
                    bic, ssr = calculate_bic(opt_prob, x, y, solver, opt_params)
                    push!(file_results, (name, opt_params, opt_sol, opt_prob, bic, ssr))
                    
                    # Plot the fit
                    plot_model_fit(x, y, opt_params, opt_sol, bic, ssr, "$filename - $name Model")
                end
            catch e
                println("   ⚠️ Failed to fit $name model: $e")
            end
        end
        
        results_20k[filename] = file_results
        
    else
        println("   → Fitting logistic growth model")
        
        # Fit simple logistic for untreated
        p_init = [0.5, 20000.0]
        bounds = [(0.0, 2.0), (1000.0, 50000.0)]
        
        try
            opt_params, opt_sol, opt_prob = setUpProblem(logistic_growth!, x, y, solver, u0, p_init, tspan, bounds)
            
            if opt_params !== nothing
                bic, ssr = calculate_bic(opt_prob, x, y, solver, opt_params)
                results_20k[filename] = [("Logistic", opt_params, opt_sol, opt_prob, bic, ssr)]
                
                # Plot the fit
                plot_model_fit(x, y, opt_params, opt_sol, bic, ssr, "$filename - Logistic Growth")
            end
        catch e
            println("   ⚠️ Failed to fit logistic model: $e")
        end
    end
end

In [ ]:
# Load and analyze 30k seeding density data
println("\nLoading 30k seeding density data...")
datasets_30k = load_intermittent_data("30k")

# Initialize storage for results
results_30k = Dict{String, Any}()
xy_data_30k = Dict{String, Tuple{Vector{Float64}, Vector{Float64}}}()

# Process each dataset (same logic as 20k)
for (filename, df) in datasets_30k
    println("\n📊 Processing: $filename")
    
    # Extract x,y data
    x, y = extract_xy_from_dayaverages(df)
    xy_data_30k[filename] = (x, y)
    
    # Set up common parameters
    solver = Rodas5()
    tspan = (x[1], x[end])
    u0 = [y[1]]
    
    # Determine if treated or untreated
    is_treated = occursin("T_", filename) && !occursin("UT_", filename)
    
    if is_treated
        println("   → Fitting treatment models")
        
        # Try different treatment models with adjusted bounds for 30k
        models_to_try = [
            (logistic_exp_decay!, [(0.0, 2.0), (100.0, 60000.0), (0.0, 0.9), (0.5, 10.0)], "Exponential Decay"),
            (intermittent_treatment!, [(0.0, 2.0), (100.0, 60000.0), (0.0, 0.9), (1.0, 7.0), (1.0, 7.0)], "Intermittent"),
            (logistic_hill_drug!, [(0.0, 2.0), (100.0, 60000.0), (0.0, 0.9), (1.0, 5.0), (1.0, 10.0)], "Hill Response")
        ]
        
        file_results = []
        
        for (model, bounds, name) in models_to_try
            p_init = [mean(b) for b in bounds]
            
            try
                opt_params, opt_sol, opt_prob = setUpProblem(model, x, y, solver, u0, p_init, tspan, bounds)
                
                if opt_params !== nothing
                    bic, ssr = calculate_bic(opt_prob, x, y, solver, opt_params)
                    push!(file_results, (name, opt_params, opt_sol, opt_prob, bic, ssr))
                    
                    # Plot the fit
                    plot_model_fit(x, y, opt_params, opt_sol, bic, ssr, "$filename - $name Model")
                end
            catch e
                println("   ⚠️ Failed to fit $name model: $e")
            end
        end
        
        results_30k[filename] = file_results
        
    else
        println("   → Fitting logistic growth model")
        
        # Fit simple logistic for untreated with adjusted bounds for 30k
        p_init = [0.5, 30000.0]
        bounds = [(0.0, 2.0), (1000.0, 60000.0)]
        
        try
            opt_params, opt_sol, opt_prob = setUpProblem(logistic_growth!, x, y, solver, u0, p_init, tspan, bounds)
            
            if opt_params !== nothing
                bic, ssr = calculate_bic(opt_prob, x, y, solver, opt_params)
                results_30k[filename] = [("Logistic", opt_params, opt_sol, opt_prob, bic, ssr)]
                
                # Plot the fit
                plot_model_fit(x, y, opt_params, opt_sol, bic, ssr, "$filename - Logistic Growth")
            end
        catch e
            println("   ⚠️ Failed to fit logistic model: $e")
        end
    end
end

In [ ]:
# Summary and comparison of results
function summarize_results(results_dict, seeding_density)
    println("\n" * "="^60)
    println("SUMMARY: $seeding_density Seeding Density Results")
    println("="^60)
    
    summary_data = []
    
    for (filename, file_results) in results_dict
        println("\n📁 $filename:")
        
        if !isempty(file_results)
            # Sort by BIC (lower is better)
            sorted_results = sort(file_results, by = x -> x[5])
            
            for (i, (model_name, params, sol, prob, bic, ssr)) in enumerate(sorted_results)
                rank_symbol = i == 1 ? "🥇" : i == 2 ? "🥈" : i == 3 ? "🥉" : "  "
                println("  $rank_symbol $model_name: BIC = $(round(bic, digits=2)), SSR = $(round(ssr, digits=6))")
                println("     Parameters: $(round.(params, digits=4))")
                
                push!(summary_data, (
                    SeeddingDensity = seeding_density,
                    File = filename,
                    Model = model_name,
                    BIC = bic,
                    SSR = ssr,
                    Parameters = join(round.(params, digits=4), ", "),
                    Rank = i
                ))
            end
        else
            println("  ❌ No successful fits")
        end
    end
    
    return summary_data
end

# Generate summaries
summary_20k = summarize_results(results_20k, "20k")
summary_30k = summarize_results(results_30k, "30k")

# Combine and save results
all_summaries = vcat(summary_20k, summary_30k)
summary_df = DataFrame(all_summaries)

# Save to CSV
CSV.write("intermittent_modeling_results.csv", summary_df)
println("\n💾 Results saved to: intermittent_modeling_results.csv")

In [ ]:
# Comparative plots: 20k vs 30k seeding densities
function create_comparative_plots()
    # Compare untreated growth between seeding densities
    plt_untreated = plot(title="Untreated Growth: 20k vs 30k Seeding", xlabel="Day", ylabel="Cell Count")
    
    # Plot 20k untreated data
    for filename in keys(xy_data_20k)
        if occursin("UT_", filename)
            x, y = xy_data_20k[filename]
            plot!(plt_untreated, x, y, label="20k - $filename", marker=:circle, lw=2)
        end
    end
    
    # Plot 30k untreated data
    for filename in keys(xy_data_30k)
        if occursin("UT_", filename)
            x, y = xy_data_30k[filename]
            plot!(plt_untreated, x, y, label="30k - $filename", marker=:square, lw=2, linestyle=:dash)
        end
    end
    
    display(plt_untreated)
    
    # Compare treated growth between seeding densities
    plt_treated = plot(title="Treated Growth: 20k vs 30k Seeding", xlabel="Day", ylabel="Cell Count")
    
    # Plot 20k treated data
    for filename in keys(xy_data_20k)
        if occursin("T_", filename) && !occursin("UT_", filename)
            x, y = xy_data_20k[filename]
            plot!(plt_treated, x, y, label="20k - $filename", marker=:circle, lw=2)
        end
    end
    
    # Plot 30k treated data
    for filename in keys(xy_data_30k)
        if occursin("T_", filename) && !occursin("UT_", filename)
            x, y = xy_data_30k[filename]
            plot!(plt_treated, x, y, label="30k - $filename", marker=:square, lw=2, linestyle=:dash)
        end
    end
    
    display(plt_treated)
end

create_comparative_plots()

## Analysis Summary

This notebook provides focused mathematical modeling for intermittent treatment data:

### Key Features:
1. **Organized Data Loading**: Directly loads from your processed datasets with exact seeding densities
2. **Multiple Model Types**: 
   - Standard logistic growth (untreated)
   - Exponential drug decay model
   - Intermittent treatment model (step function)
   - Hill function drug response
3. **Comparative Analysis**: Side-by-side comparison of 20k vs 30k seeding densities
4. **Model Selection**: BIC-based ranking to identify best-fitting models
5. **Results Export**: Comprehensive CSV output for further analysis

### Data Structure:
- Loads from: `Processed_Datasets/Intermittent Data/{20k,30k}_seeding_density/Averages/`
- Processes: Day-averaged cell count data
- Handles: Both naive (A2780) and resistant (A2780cis) cell lines
- Compares: Treated vs untreated conditions

### Output:
- Individual model fits with plots
- BIC comparison for model selection
- Summary CSV file with all parameters
- Comparative visualizations